In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import os
from dotenv import load_dotenv
import json
from litellm import completion

In [ ]:
model_name = "llama-3.2-3b-it-Perovskite-PaperExtractor"
# model_name = "meta-llama/Llama-3.2-3B-Instruct"
device = "cuda"

model = AutoModelForCausalLM.from_pretrained(model_name, ).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.model_max_length = 60000
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    temperature=0.1,
    device=0
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.81s/it]


In [ ]:
PREFIX = """
You are a helpful scientific assistant. Your task is to extract relevant scientific data from the provided text about perovskite solar cells and passivating molecules. If the data is not available in the text, return null for the respective fields. Output the information in JSON format with the following fields:
- `control_pce`: Power conversion efficiency for control perovskite (numeric).
- `control_voc`: Open-circuit voltage for control perovskite (numeric).
- `treated_pce`: Best Power conversion efficiency for treated perovskite (numeric).
- `treated_voc`: Best Open-circuit voltage for treated perovskite (numeric).
- `passivating_molecule`: Name of the champion passivating molecule tested.
- `perovskite_composition`: Chemical formula of the perovskite (string).
- `electron_transport_layer`: Material used as the electron transport layer (string).
- `hole_transport_layer`: Material used as the hole transport layer (string).
- Stability tests: Include any stability tests mentioned. Stability tests can be done in dark storage, light-soaking, thermal cycling, light cycling, and solar-thermal cycling. If none of these types are tested, do not include a JSON object for them.
For each test, the value should follow this format:
```json
{
  "test_name": null,
  "temperature": null,
  "time": null,
  "humidity": null,
  "control_efficiency": null,
  "treatment_efficiency": null
}

The JSON structure must follow this exact format:
{
  "control_pce": null,
  "control_voc": null,
  "treated_pce": null,
  "treated_voc": null,
  "passivating_molecule": null,
  "perovskite_composition": null,
  "electron_transport_layer": null,
  "hole_transport_layer": null,
  "stability_tests": [
    {
      "test_name": null,
      "temperature": null,
      "time": null,
      "humidity": null,
      "control_efficiency": null,
      "treatment_efficiency": null
    },
  ]
}
Be concise and accurate. Include only information explicitly present in the text.
Don't return ranges for any values, as this will cause the JSON to not parse correctly. If a range is presented, return the range as a string.
Only return JSON. The text is below:
"""
SUFFIX = """\n\n{sample}\n\n"""
def create_prompt(system, user):
    tokens = tokenizer.encode(user, max_length=25000, truncation=True)
    truncated_user = tokenizer.decode(tokens)
    return [
    {"role": "system", "content": system},
    {"role": "user", "content": truncated_user}, ]

In [ ]:
contents = contents # {}

In [24]:
txt_dir = "../../data/txts"
# contents = {}
for filename in os.listdir(txt_dir):
    if filename.endswith(".txt") == False:
        continue
    filepath = os.path.join(txt_dir, filename)
    paper_id = os.path.splitext(filename)[0]
    if paper_id in contents:
        continue
    print(paper_id)

    with open(filepath, "r", encoding="utf-8") as file:
        text = file.read()
        instruction = create_prompt(PREFIX, text)
        json_string = pipe(instruction, max_new_tokens=1024)[0]["generated_text"][-1]['content']
        clean_json_string = json_string.strip("```json").strip("```").strip()
        try:
            parsed_data = json.loads(clean_json_string)
            contents[paper_id] = parsed_data
        except json.JSONDecodeError as e:
            print("Error creating JSON", e)
            print(clean_json_string)
        

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


132


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Error creating JSON Expecting value: line 1 column 1 (char 0)
Here is the extracted data in JSON format:

```
{
  "control_pce": 8.7,
  "control_voc": 1.13,
  "treated_pce": 23.1,
  "treated_voc": 1.177,
  "passivating_molecule": "1-dodecanethiol (DDT)",
  "perovskite_composition": "Cs0.05 FA x MA 1 - x PbI 3",
  "electron_transport_layer": "Spiro-OMeTAD",
  "hole_transport_layer": "Spiro-OMeTAD",
  "stability_tests": [
    {
      "test_name": "Wetting stability",
      "temperature": null,
      "time": 530,
      "humidity": 50-65,
      "control_efficiency": 19.7,
      "treatment_efficiency": 19.6
    },
    {
      "test_name": "Thermal stability",
      "temperature": 50-85,
      "time": 144,
      "humidity": null,
      "control_efficiency": 20.2,
      "treatment_efficiency": 20.3
    },
    {
      "test_name": "Illumination stability",
      "temperature": 30-35,
      "time": 2000,
      "humidity": null,
      "control_efficiency": 19.0,
      "treatment_efficiency": 20.

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

60


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


120


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


67


In [25]:
json_path = "../../data/finetuned_llama_output.json"
with open(json_path, "w") as file:
    json.dump(contents, file, indent=4)

{'control_pce': None,
 'control_voc': None,
 'treated_pce': 23.9,
 'treated_voc': None,
 'passivating_molecule': 'isopropylammonium chloride',
 'perovskite_composition': 'FAPbI3',
 'electron_transport_layer': None,
 'hole_transport_layer': None,
 'stability_tests': [{'test_name': 'dark storage',
   'temperature': None,
   'time': None,
   'humidity': None,
   'control_efficiency': None,
   'treatment_efficiency': None},
  {'test_name': 'light-soaking',
   'temperature': None,
   'time': None,
   'humidity': None,
   'control_efficiency': None,
   'treatment_efficiency': None},
  {'test_name': 'thermal cycling',
   'temperature': None,
   'time': None,
   'humidity': None,
   'control_efficiency': None,
   'treatment_efficiency': None},
  {'test_name': 'light cycling',
   'temperature': None,
   'time': None,
   'humidity': None,
   'control_efficiency': None,
   'treatment_efficiency': None},
  {'test_name': 'solar-thermal cycling',
   'temperature': None,
   'time': None,
   'humidity